In [2]:
from flask import Flask, jsonify, request
import pandas as pd
import numpy as np
import joblib
import time

In [6]:
app = Flask(__name__)

In [8]:
# prepare input for base learners
def prepare_input(X_q):
    """This function preprocess the data and construct the input features for base learners 
    using trained scaler for numerical features and one hot encoder for categorical features """
    
    # define the categorical & numerical features which are taken for training the model
    cat_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
    num_features = ['age', 'duration', 'campaign', 'pdays', 'previous', 'cons.price.idx', 'cons.conf.idx', 'nr.employed']
    
    # load standard scaler from pickle file
    scaler = joblib.load('num_features_scaler.pkl')
    # load trained onehotencoder for categorical features
    cat_features_ohe = joblib.load('cat_features_ohe.pkl')
    
    # tranform the numerical features with trained standard scaler and convert it into a dataframe for further use
    X_q_num = pd.DataFrame(scaler.transform(X_q[num_features]), columns=num_features)
    # print(X_q_num, type(X_q_num))
    
    # tranform the categorical features with trained onehotencoder and convert it into a dataframe for further use
    X_q_ohe = cat_features_ohe.transform(X_q[cat_features])
    # print(X_q_ohe)
    cat_feature_labels_ohe = np.concatenate(cat_features_ohe.categories_).ravel().tolist()
    X_q_ohe = pd.DataFrame(X_q_ohe.toarray(), columns=cat_feature_labels_ohe)
    # print(type(X_q_ohe), type(X_q_num))
    
    # merge both numerical feature set and categorical feature set
    X_q_final = pd.concat([X_q_ohe, X_q_num], axis=1)
    
    # print(X_q_final)
    return X_q_final

In [ ]:
@app.route('/predict', methods=['POST'])
def final_predict(X_q, y=''):
    """This function predcts target label for query instance, the prediction is from trained meta model """    
    
    start_time = time.time()

    # get the form inputs through request
    form_inputs = request.form.to_dict()
    print(form_inputs)
    
    return True
    
    # data pre processing and prepare the input to predict through meta model
    X_input = prepare_input(X_q)
    
    # load the trained base models from pickle file
    base_models = joblib.load('base_learners.pkl')
    # print(base_models)
    
    # initiate list for storing the predictions from each base learner for given query point
    input_for_meta = []
    # predictions from base learners
    for model in base_models:
        input_for_meta.append(model.predict(X_input))
        
    # construct the input to meta model from base learners predictions
    input_for_meta = np.transpose(np.asarray(input_for_meta))
    # load trained meta model from pickle file 
    meta_model = joblib.load('meta_model.pkl')
    # final prediction
    final_prediction = meta_model.predict(input_for_meta)
    
    # print(final_prediction)
    if final_prediction:
        prediction = 'Positive'
    else:
        prediction = 'Negative'
    
    end_time = time.time()
    
    # computing the time taken for predicting the target label
    time_for_prediction = str(end_time - start_time) + ' seconds'
    
    # handling true label
    y = 'Positive' if y==1 else 'Negative'
    
    response = jsonify({'prediction': prediction, 'True label': y, 'Time taken for Prediction': time_for_prediction})
    # return predicted label, time taken for predicting and the original target label if passed as input
    return response

In [10]:
@app.route('/index')
def index():
    return flask.render_template('index.html')

In [13]:
@app.route('/')
def welcome():
    return 'Welcome to the Lead Score Prediction home page'

In [12]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5050)

In [5]:
data = pd.read_csv('bank-additional-full.csv', ';')

In [8]:
data.previous

0        0
1        0
2        0
3        0
4        0
        ..
41183    0
41184    0
41185    0
41186    0
41187    1
Name: previous, Length: 41188, dtype: int64

In [35]:
req_form = {'name': 'Prasad', 'age': '34', 'job': 'blue-collar', 'marital': 'married', 'education': 'professional.course', 'default': 'no', 'housing': 'no', 'loan': 'yes', 'contact': 'cellular', 'month': 'mar', 'day_of_week': 'fri', 'duration': '600', 'campaign': '2', 'pdays': '999', 'previous': '0', 'poutcome': 'nonexistent', 'emp.var.rate': '0.081886', 'cons.price.idx': '93.575664', 'cons.conf.idx': '-40.502600', 'nr.employed': '5167.035911'}
# print(list(req_form.values()))
# print(req_form.keys())
# print(np.asarray(list(req_form.values())).reshape(1,-1).shape)
X_q = pd.DataFrame(np.asarray(list(req_form.values())).reshape(1,-1), columns=req_form.keys())
X_q.drop('name', axis=1, inplace=True)
(X_q)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,nr.employed
0,34,blue-collar,married,professional.course,no,no,yes,cellular,mar,fri,600,2,999,0,nonexistent,0.081886,93.575664,-40.502600,5167.035911
